# IMPACT paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from liriscat import utils
utils.set_seed(0)
from liriscat import dataset
from liriscat import model

import logging
import gc
import json
import torch
import pandas as pd
from importlib import reload

CUDA is not available. Skipping CUDA seed setting.


/home/arthurb/Programmation/liriscat/y/envs/liriscat-env/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


#### 1.2. Set up the loggers (recommended)

In [3]:
utils.setuplogger(verbose = True, log_name="liriscat")

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [4]:
import warnings
import numpy as np

gc.collect()
torch.cuda.empty_cache()

reload(utils)
reload(model)
reload(dataset)


<module 'liriscat.dataset' from '/home/arthurb/Programmation/liriscat/liriscat/dataset/__init__.py'>

In [5]:
config = utils.generate_eval_config(esc = 'error', valid_metric= 'rmse', pred_metrics = ['rmse', 'mae'], profile_metrics = ['doa', 'pc-er', 'rm'], save_params=False)
utils.set_seed(config["seed"])

config["dataset_name"] = "assist0910"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.02026
config['lambda'] = 1.2e-5
config['d_in'] = 4
config['num_responses'] = 12
#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 25:52] assist0910


In [7]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'+str(config["dataset_name"])
config['params_path']='../ckpt/'+str(config["dataset_name"])

pred_metrics = {m:[] for m in config['pred_metrics']}
profile_metrics = {m:[] for m in config['profile_metrics']}

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))

[INFO 27:22] #### assist0910 ####
[INFO 27:22] #### config : {'seed': 0, 'dataset_name': 'assist0910', 'load_params': False, 'save_params': False, 'embs_path': '../embs/assist0910', 'params_path': '../ckpt/assist0910', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'rmse', 'learning_rate': 0.02026, 'batch_size': 2048, 'num_epochs': 200, 'eval_freq': 1, 'patience': 30, 'device': device(type='cuda'), 'lambda': 1.2e-05, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['rmse', 'mae'], 'profile_metrics': ['doa', 'pc-er', 'rm'], 'num_responses': 12, 'low_mem': False, 'd_in': 4} ####


In [ ]:
i_fold = 0
## Dataframe columns : (user_id, question_id, response, category_id)
train_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
valid_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
test_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})

In [35]:
reload(dataset)
train_data = dataset.Dataset(train_df, concept_map, metadata, config)
# valid_data = dataset.LoaderDataset(valid_quadruplets, concept_map, metadata)
# test_data = dataset.LoaderDataset(test_quadruplets, concept_map, metadata)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [17]:
train_data.n_logs

123494

In [ ]:
import numpy as np

def split_by_u_q_optimized(u, q, random_state=None):
    """
    Splits data tuples into two groups (e.g., query and meta) without separating tuples
    that share the same (u, q) pair.

    Optimized version: sorts the data by u so that each user's indices are contiguous,
    then processes each block to assign a random group label per unique q.

    Parameters
    ----------
    u : array-like
        Array of user values.
    q : array-like
        Array of question values.
    random_state : int or None, optional
        Seed for reproducibility.

    Returns
    -------
    groups : np.ndarray
        Array of group labels (0 or 1) for each data tuple.
    """
    # Convert to numpy arrays
    u = np.asarray(u)
    q = np.asarray(q)
    groups = np.empty(u.shape[0], dtype=np.int8)

    rng = np.random.default_rng(random_state)

    # Sort indices by u so that rows for the same user are contiguous.
    sorted_u_idx = np.argsort(u)
    u_sorted = u[sorted_u_idx]

    # Find the boundaries for each unique user.
    unique_users, start_idx = np.unique(u_sorted, return_index=True)
    end_idx = np.append(start_idx[1:], len(u_sorted))

    # Process each contiguous block corresponding to a single user.
    for s, e in zip(start_idx, end_idx):
        block_idx = sorted_u_idx[s:e]  # indices for one user
        q_block = q[block_idx]
        # For this user's block, get unique q values and an inverse index
        unique_q, inv = np.unique(q_block, return_inverse=True)
        # Randomly assign a group label (0 or 1) for each unique q in this block
        group_assignments = rng.choice([0, 1], size=len(unique_q), p=[0.8,0.2])
        # Map the group labels back to the original indices in the block
        groups[block_idx] = group_assignments[inv]

    return groups

# Example usage:
# Let's assume train[0] is a structured array/dict-like with keys 'user_id' and 'item_id'.
# For demonstration, we simulate some sample data:

import time
start = time.time()

for i in range(1):
    group_labels = split_by_u_q_optimized(train[0]['user_id'], train[0]['item_id'], random_state=42)

    # Now, all tuples with the same (user_id, item_id) pair have the same group label.
    query_mask = (group_labels == 0)
    meta_mask  = (group_labels == 1)

    query_users = train[0]['user_id'][query_mask].to_numpy()
    query_questions = train[0]['item_id'][query_mask].to_numpy()
    meta_users = train[0]['user_id'][meta_mask].to_numpy()
    meta_questions = train[0]['item_id'][meta_mask].to_numpy()
    #print(s2)
print("Total time:", time.time() - start)
